In [1]:
import json

dataset_path = "data/lean_workbook.json"
with open(dataset_path, "r") as f:
    dataset = json.load(f)
print(type(dataset))
print(len(dataset))
print(dataset[0].keys())
print(dataset[0]["proof"])
proof_problems = [sample for sample in dataset if len(sample["proof"]) > 0]
print(len(proof_problems))

<class 'list'>
140214
dict_keys(['natural_language_statement', 'answer', 'tags', 'formal_statement', 'split', 'proof'])
[]
12823


In [5]:
print(json.dumps(dataset[0], indent=2))
print(json.dumps(proof_problems[1], indent=2))

{
  "natural_language_statement": "Let $a,b,c$ be positive real numbers .Prove that $ \\frac{b+c}{\\sqrt{a^2 + 8bc}} + \\frac{c+a}{\\sqrt{b^2 + 8ca}} + \\frac{a+b}{\\sqrt{c^2 + 8ab}} \\geq 2. $",
  "answer": "2",
  "tags": [
    "inequality"
  ],
  "formal_statement": "theorem lean_workbook_0 (a b c : \u211d) (ha : 0 < a) (hb : 0 < b) (hc : 0 < c) : (b + c) / Real.sqrt (a ^ 2 + 8 * b * c) + (c + a) / Real.sqrt (b ^ 2 + 8 * c * a) + (a + b) / Real.sqrt (c ^ 2 + 8 * a * b) \u2265 2  :=  by sorry",
  "split": "lean_workbook",
  "proof": []
}
{
  "natural_language_statement": "Prove that $x-1 \\geq \\ln x$ for $x>0$.",
  "answer": "",
  "tags": [
    "inequality",
    "logarithm",
    "calculus"
  ],
  "formal_statement": "theorem lean_workbook_26 (x : \u211d) (hx : 0 < x) : x - 1 \u2265 Real.log x  :=  by sorry",
  "split": "lean_workbook",
  "proof": [
    "have h1 : 0 \u2264 (x - 1)^2 := sq_nonneg (x - 1)\nnlinarith [log_le_sub_one_of_pos hx]",
    "nlinarith [log_le_sub_one_of_pos hx]"

In [ ]:
from datasets import load_dataset
dataset = load_dataest("hoskinson-center/proofnet")  # lean4 version of proofnet

In [1]:
# download MATH
from datasets import load_dataset
dataset = load_dataset("hendrycks/competition_math")["train"]
print(dataset)
print(type(list(dataset)))
print(list(dataset)[0])

Dataset({
    features: ['problem', 'level', 'type', 'solution'],
    num_rows: 7500
})
<class 'list'>
{'problem': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).', 'level': 'Level 5', 'type': 'Algebra', 'solution': 'For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.'}


In [1]:
from library import build_library_from_outputs
output_path = "outputs/qwen_MATH_lemma_extract.jsonl"
lemmas = build_library_from_outputs(output_path)

Found 7500 samples
Extracted 1461 lemmas


In [2]:
from utils import dump_jsonl
from dataclasses import asdict
save_path = "libraries/qwen_lemmas.jsonl"
dump_jsonl([asdict(lemma) for lemma in lemmas], save_path)

In [7]:
# extract lemma requests
from utils import dump_jsonl
from processing import extract_relevant_lemmas_from_outputs
outputs_path = "outputs/test_lemma_descriptions.jsonl"
samples = extract_relevant_lemmas_from_outputs(outputs_path)
samples = [s for s in samples if s["lemma_requests"] is not None and len(s["extracted_lemma_requests"]) > 0]
dump_jsonl(samples, "outputs/test_extracted_lemma_descriptions.jsonl")

Found 1552 samples
Found 6795 requests


In [1]:
# pair samples by computing pairwise cosine similarities between 
from library import compute_batched_similarity
from utils import load_jsonl
import torch
sources = load_jsonl("outputs/test_extracted_lemma_descriptions_embeddings.jsonl")
targets = load_jsonl("libraries/qwen_lemmas_embeddings.jsonl")
sources = torch.tensor([embedding for sample in sources for embedding in sample["embeddings"]])
targets = torch.tensor([sample["embedding"][0] for sample in targets])
sims_dict = compute_batched_similarity(sources, targets, batch_size=128)

100%|██████████| 151/151 [01:02<00:00,  2.40it/s]


In [4]:
print(len(sims_list))

6795
